In [1]:
# !pip install git+https://github.com/Navu45/image-merging.git
# !pip install git+https://github.com/ChaoningZhang/MobileSAM.git

In [19]:
import gradio as gr
import numpy as np
import torch
from PIL import Image
from diffusers import DDIMScheduler
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator
from torchvision.transforms.functional import to_pil_image

from models.inpaint_image import InpaintPipeline

In [3]:
!git lfs clone https://huggingface.co/dhkim2810/MobileSAM

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
fatal: destination path 'MobileSAM' already exists and is not an empty directory.
Error(s) during clone:
git clone failed: exit status 128


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = InpaintPipeline.from_pretrained(
    'Fantasy-Studio/Paint-by-Example',
    torch_dtype=torch.float16,
    # local_files_only=True
)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()

model_type = "vit_t"
sam_checkpoint = "./MobileSAM/mobile_sam.pt"
mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()
mask_generator = SamAutomaticMaskGenerator(mobile_sam)

unet\diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You are using a model of type clip_vision_model to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [25]:
import cv2, numpy as np


def generate_mask(source_image, target_image, pipe):
    source_embeds = pipe.encode_image(source_image)
    target_embeds = pipe.encode_image(target_image)
    negative_mask_image = pipe.generate_mask(source_image,
                                             height=source_image.height,
                                             width=source_image.width,
                                             target_prompt_embeds=target_embeds,
                                             source_prompt_embeds=source_embeds,
                                             num_maps_per_mask=10,
                                             mask_encode_strength=0.5,
                                             mask_thresholding_ratio=3.0,
                                             num_inference_steps=50,
                                             guidance_scale=1)
    negative_mask_image = to_pil_image(negative_mask_image, 'L')
    negative_mask_image = cv2.medianBlur(np.asarray(negative_mask_image), 59)
    return to_pil_image(negative_mask_image, 'L')

In [ ]:
def save_images(*images):
    for i, image in enumerate(images):
        image.save(f'image{i}.png')

def create_img(source_image, target_image) -> list:
    source_image, target_image = source_image.resize((512, 512)), target_image.resize((512, 512))

    return pipe(image=source_image,
                mask_image=generate_mask(source_image, target_image, pipe),
                example_image=target_image,
                num_inference_steps=100).images[0]

with gr.Blocks() as demo:
    with gr.Tab(""):
        with gr.Row():
            source_image_input = gr.Image(type='pil')
            target_image_input = gr.Image(type='pil')
        with gr.Row():
            image_output = gr.Image()
        image_button = gr.Button("Generate image!")
    image_button.click(create_img, inputs=[source_image_input, target_image_input], outputs=image_output)

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]